In [17]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.fft import fft
import pandas as pd
import glob
import re

# https://pages.mtu.edu/~suits/notefreqs.html
# table of musical notes and respective frequencies.

In [36]:
def get_freqsig(wav):
    # if i is None:
    #     i=0
    samplerate, data = wavfile.read(wav)
    data= data.reshape(2, -1)
    n = data.shape[1]
    t = n/samplerate
    dt = t/n
    freq = np.arange(n)/t #+ i*1
    mono = data[0] # Mono sound -- channel two is stereo.
    fhat = fft(mono, n)
    psd = (fhat*np.conj(fhat) / n).astype(float)  

    psd_max = psd.max()
    psd_min = psd.min()
    psd_norm = (psd-psd_min) / (psd_max-psd_min)
    
    tones = freq[psd_norm > 0.1]
    tone_psd = psd_norm[psd_norm > 0.1]

    df = pd.DataFrame({'tones':tones, 'psd_norm':tone_psd})
    df = df.loc[df.tones<5000]
    
    # df.tones = df.tones.apply(lambda x: np.floor(x))  # intially used to elimate spread around a given frequency. however, this might cause significance of some frequencies to be overlooked
    # df = df.groupby('tones').psd_norm.max().reset_index()
    
    
    tone_psd_dix = {tone:psd for (tone,psd) in zip(df.tones, df.psd_norm)}
    return tone_psd_dix

In [39]:
note_paths = glob.glob(r"guitar_notes/acoustic/string*/*.wav")
note_paths

note_dix = {}
pattern = re.compile(r"(\d+)(?=.wav)")
for path in note_paths:
    string_fret = pattern.search(path).group() 
    string_no = str(string_fret)[0]
    fret = int(str(string_fret)[1:])
    tones = get_freqsig(path)
    if string_no not in note_dix.keys():
        note_dix[string_no] = {fret: tones}
    break


C:\Users\callu\AppData\Local\Temp\ipykernel_1692\199672267.py:12: ComplexWarning: Casting complex values to real discards the imaginary part
  psd = (fhat*np.conj(fhat) / n).astype(float)


In [40]:
note_dix

{'1': {0: {43.445692883895134: 0.13590924393970097,
   44.19475655430712: 0.14463641342260028,
   44.56928838951311: 0.11978906693269584,
   44.943820224719104: 0.11559290744952802,
   45.3183520599251: 0.1496212568898807,
   46.06741573033708: 0.1528977392646454,
   46.441947565543074: 0.19886119842711097,
   46.81647940074907: 0.10804958151430845,
   47.19101123595506: 0.12595566937935937,
   47.56554307116105: 0.2642043057465772,
   47.940074906367045: 0.21789246549809976,
   48.31460674157304: 0.13017872744334547,
   48.68913857677903: 0.36142746188089564,
   49.06367041198502: 0.3056954766244926,
   49.438202247191015: 0.16459936777529746,
   49.81273408239701: 0.1895960573278466,
   50.187265917603: 0.537514885538841,
   50.56179775280899: 0.17736285433931767,
   50.936329588014985: 0.3998703436768465,
   51.31086142322098: 0.5518990570178005,
   51.68539325842697: 0.2678727060855854,
   52.05992509363296: 0.38317505531373286,
   52.434456928838955: 0.44316422626829916,
   52.808